In [1]:
import os
import numpy as np

DATA_PATH = os.path.join('datasets3')

no_sequences = 120
sequence_length = 30
actions = np.array(['나', '짜증', '너', '사감', '우유', '무섭다', '청소당번', '가다', '배고프다', 
                    '나이', '기분', '때문', '좋다', '배부르다', '밥', 
                    '덥다', '동대문', '춥다', '형편없다', '차다', '온도', '불복종', '오늘', '어디'])

In [2]:
label_map = {label:num for num, label in enumerate(actions)}

In [3]:
res = np.load(os.path.join(DATA_PATH, "불복종", "55", "0.npy"))
res

array([ 5.25242090e-01,  1.56234980e-01, -4.12383676e-01,  9.99935210e-01,
        5.40889680e-01,  1.28819734e-01, -3.73706728e-01,  9.99909282e-01,
        5.50635695e-01,  1.29900500e-01, -3.73643994e-01,  9.99897957e-01,
        5.58283329e-01,  1.31436422e-01, -3.73581737e-01,  9.99912679e-01,
        5.11326849e-01,  1.28979847e-01, -3.76141697e-01,  9.99906361e-01,
        5.00950992e-01,  1.30386218e-01, -3.76034170e-01,  9.99898672e-01,
        4.91845518e-01,  1.31942198e-01, -3.76209646e-01,  9.99906361e-01,
        5.69337428e-01,  1.52190030e-01, -1.51682362e-01,  9.99892831e-01,
        4.83698070e-01,  1.56300530e-01, -1.61550820e-01,  9.99892294e-01,
        5.45036614e-01,  1.93678945e-01, -3.32308203e-01,  9.99956131e-01,
        5.06362140e-01,  1.94103912e-01, -3.37412238e-01,  9.99953568e-01,
        6.49293900e-01,  3.62650692e-01, -5.75155206e-02,  9.99914587e-01,
        4.30100292e-01,  3.41670841e-01, -1.44255668e-01,  9.99829113e-01,
        6.91555798e-01,  

In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=True)
            if res.dtype == object:
                print(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
np.shape(sequences)
np.array(sequences, dtype=np.float32)

array([[[ 5.67880034e-01,  8.87141824e-02, -7.04681456e-01, ...,
          4.35236841e-01,  8.63011777e-01,  2.61303782e-03],
        [ 5.80098510e-01,  8.87267888e-02, -5.02196670e-01, ...,
          4.40952629e-01,  8.61321449e-01, -5.87789051e-04],
        [ 5.83624721e-01,  8.86844918e-02, -4.06311601e-01, ...,
          4.41308618e-01,  8.60776126e-01,  7.49777188e-04],
        ...,
        [ 5.80093682e-01,  8.85837078e-02, -3.36367518e-01, ...,
          4.85324025e-01,  5.06578743e-01, -1.05407685e-02],
        [ 5.80223739e-01,  8.81564617e-02, -3.34368110e-01, ...,
          4.79757160e-01,  5.34719944e-01, -9.25555639e-03],
        [ 5.80423236e-01,  8.61060619e-02, -3.35935324e-01, ...,
          4.73128766e-01,  6.30794525e-01, -1.50243705e-02]],

       [[ 5.81067383e-01,  8.50973129e-02, -3.37135792e-01, ...,
          4.69907045e-01,  7.15622187e-01, -1.47341397e-02],
        [ 5.81037581e-01,  8.30873027e-02, -3.55168194e-01, ...,
          4.58682775e-01,  8.47673178e

In [5]:
X = np.array(sequences)
labels = np.array(labels)
print(labels)

[ 0  0  0 ... 23 23 23]


In [6]:
labels.shape

(2880,)

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.nn import leaky_relu
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

In [8]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [9]:
label = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.05)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2736, 30, 267), (144, 30, 267), (2736, 24), (144, 24))

In [10]:
from tensorflow.keras.callbacks import Callback

class callback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        print(logs)
        if(logs.get('acc') > 0.999999):
            print("\n-------------------reach 100% accuracy----------------------")
            self.model.stop_training = True
            
callbacks = callback()

In [11]:
try:
    os.makedir('models120_1')
except:
    pass

model_path = os.path.join('models120_1', 'model.h5')
    
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(30, 267)),
    BatchNormalization(),
    LeakyReLU(alpha=0.3),
    LSTM(256, return_sequences=False),
    BatchNormalization(),
    LeakyReLU(alpha=0.3),
#     LSTM(64, return_sequences=False),
#     BatchNormalization(),
#     LeakyReLU(alpha=0.3),
    Dense(512),
    BatchNormalization(),
    LeakyReLU(alpha=0.3),
    Dense(256),
    BatchNormalization(),
    LeakyReLU(alpha=0.3),
    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.3),
    Dense(24, activation='softmax')
])

model.compile(optimizer=Adam(),
              loss='CategoricalCrossentropy',
              metrics=['acc'])

EPOCHS = 200
BATCHSIZE = 64

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCHSIZE, callbacks=[callbacks])
model.save(model_path)

Epoch 1/200
43/43 [==============================] - 8s 105ms/step - loss: 1.6996 - acc: 0.5008
{'loss': 1.0984982252120972, 'acc': 0.672149121761322}
Epoch 2/200
43/43 [==============================] - 5s 105ms/step - loss: 0.4089 - acc: 0.8819
{'loss': 0.36509525775909424, 'acc': 0.8899853825569153}
Epoch 3/200
43/43 [==============================] - 6s 133ms/step - loss: 0.2785 - acc: 0.9163
{'loss': 0.2428525984287262, 'acc': 0.9290935397148132}
Epoch 4/200
43/43 [==============================] - 6s 148ms/step - loss: 0.1610 - acc: 0.9525
{'loss': 0.16614533960819244, 'acc': 0.9502924084663391}
Epoch 5/200
43/43 [==============================] - 5s 124ms/step - loss: 0.1533 - acc: 0.9550
{'loss': 0.1211603432893753, 'acc': 0.9671052694320679}
Epoch 6/200
43/43 [==============================] - 5s 117ms/step - loss: 0.1089 - acc: 0.9717
{'loss': 0.1071644127368927, 'acc': 0.9711257219314575}
Epoch 7/200
43/43 [==============================] - 5s 117ms/step - loss: 0.1170 - acc

In [12]:
models = load_model(model_path)
models.evaluate(X_test, y_test)

5/5 [==============================] - 1s 26ms/step - loss: 0.0028 - acc: 1.0000


[0.0027836207300424576, 1.0]